<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Data_Augmentation_and_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
!7z x /content/Parasite.7z

--2021-04-18 08:37:50--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20003284 (19M) [application/octet-stream]
Saving to: ‘Parasite.7z’

Parasite.7z         100%[===================>]  19.08M  27.2MB/s    in 0.7s    

2021-04-18 08:37:51 (27.2 MB/s) - ‘Parasite.7z’ saved [20003284/20003284]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 20003284 bytes (20 MiB)

Extracting archive: /content/Parasite.7z
--
Path = /content

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import torchvision
from PIL import Image

In [35]:
del data

In [36]:
data = pd.read_csv("/content/Dataset/all_meta_data_updated.csv")

In [37]:
data.species.replace({"Hymenolepis diminuta":"Hymenolepis spp",
                    "Hymenolepis nana":"Hymenolepis spp",
                    # "Taenia solium":"na",
                    # "Clonorchis sinensis":"na",
                    # "Trichinella sp.":"na",
                    # "B. coli":"na",
                    # "Toxoplasma gondii":"na",
                    },inplace = True)
data['phylum'].replace({'Roundworms':'Nematoda','Flatworm':'Platyhelminthes'},inplace = True)
data['class'].replace({'\tConoidasida':'Conoidasida','Tapeworms':'Cestoda'},inplace = True)
data['species'].replace({'Entamoeba histolytica]':'Entamoeba histolytica'},inplace = True)
data['form'].replace({'rings':'trophozoites','decorticated egg':'egg','ookinete':'egg'},inplace = True)
data['sample'].replace({'intestinal tissue':'intestine', 'intestine ':'intestine','sputum':'lungs'},inplace = True)

In [22]:
data['species'].value_counts()

Plasmodium falciparum          36
Plasmodium vivax               36
Plasmodium malariae            30
Cyclospora cayetanensis        30
Ascaris lumbricoides           29
Plasmodium ovale               27
Giardia duodenalis             27
Entamoeba histolytica          27
Strongyloides stercoralis      22
Dibothriocephalus latus        21
Hymenolepis spp                21
Trichuris Trichuria            19
Enterobius vermicularis        18
Toxocara canis                 17
Ancyclostoma sp.               17
Sarcocystis sp.                16
Cryptosporidium sp.            16
Plasmodium knowlesi            15
Toxoplasma gondii              13
Trichinella sp.                13
Echinococcus granulosus        12
Fasciola hepatica              11
B. coli                         9
Clonorchis sinensis             8
Dicrocoelium dendriticum        7
Taenia solium                   6
Echinococcus multilocularis     4
Strongyloides fuelleborni       3
Name: species, dtype: int64

In [38]:
data = [x for _,x in data.groupby('species') if len(x)>15]

data = pd.concat(data,ignore_index=True)

In [39]:
data = data[data['sample']!='na']
data = data[data['sample']!='liver']

In [8]:
labels = ['phylum', 'class', 'genus', 'species', 'form', 'sample']

In [40]:
train, val = train_test_split(
    data, test_size=0.25, random_state=0,stratify = data['species'])

In [41]:
for col in labels:
  print(15*'-'+col+15*'-')
  print(set(train[col].unique())-set(val[col].unique()))
  print(set(val[col].unique())-set(train[col].unique()))
  print(train[col].nunique(),val[col].nunique(),sep = ' -> ')
  print(val[col].nunique(),train[col].nunique(),sep = ' -> ')

---------------phylum---------------
set()
set()
5 -> 5
5 -> 5
---------------class---------------
set()
set()
7 -> 7
7 -> 7
---------------genus---------------
set()
set()
14 -> 14
14 -> 14
---------------species---------------
set()
set()
17 -> 17
17 -> 17
---------------form---------------
set()
set()
9 -> 9
9 -> 9
---------------sample---------------
set()
set()
6 -> 6
6 -> 6


In [44]:
for path in data['species'].unique():
  os.makedirs(f"/content/train/{path}")


for path in data['species'].unique():
  os.makedirs(f"/content/val/{path}")

In [43]:
!rm -r train val Train.7z Val.7z

In [45]:
t = []
t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomPerspective(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomVerticalFlip(p=1)]))


t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomVerticalFlip(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomPerspective(p=1)]))

In [13]:
def transform_save(data,index,folder):
  img = Image.open(data[8])
  images = [img]
  images.extend([T(img) for T in t])
  image_name = data[6].split('.')[0]
  for i, image in enumerate(images):
    image.convert('RGB').save(f"/content/{folder}/{data[3]}/{image_name}_{i}_{index}.jpg")
  values = [[data[0],data[1],data[2],data[3],data[4],data[5],f"./{folder}/{data[3]}/{image_name}_{i}_{index}.jpg"] for i in range(len(images))]
  return values


def get_dataframe(dataframe,folder = 'train'):
  new = []
  for i in dataframe.index:
    new.extend(transform_save(dataframe.loc[i,:].values,index =i,folder=folder))
  return pd.DataFrame(new,columns=['phylum','class','genus','species','form','sample','img_path'])

In [46]:
train = get_dataframe(train)

In [47]:
val = get_dataframe(val,folder = 'val')

In [48]:
train.to_csv('/content/train/train.csv',index = False)
val.to_csv('/content/val/val.csv',index = False)

In [49]:
!7z a Train.7z train
!7z a Val.7z val


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           18 folders, 1501 files, 15945546 bytes (16 MiB)

Creating archive: Train.7z

Items to compress: 1519

  0%      6% 144 + train/Ascaris lumbricoides/Ascaris_eggs_tissue_400x_BAM1_4_44.jpg                                                                             13% 240 + train/Cyclospora cayetanensis/Cyclospora_AF_AZ1_0_75.jpg                                                                    22% 371 + train/Dibothriocephalus latus/Diph_proglo

In [50]:
labels = ['phylum', 'class', 'genus', 'species', 'form', 'sample']
train = pd.read_csv("/content/train/train.csv")
# train = train[train['sample']!='na']
# train = train[train['sample']!='liver'] 
val = pd.read_csv("/content/val/val.csv")
# val = val[val['sample']!='na']
# val = val[val['sample']!='liver']
for col in labels:
  print(15*'-'+col+15*'-')
  print(train[col].nunique(),val[col].nunique(),sep=' -> ')

---------------phylum---------------
5 -> 5
---------------class---------------
7 -> 7
---------------genus---------------
14 -> 14
---------------species---------------
17 -> 17
---------------form---------------
9 -> 9
---------------sample---------------
6 -> 6
